# 模型训练

In [2]:
import pickle, sys, gc, time, os
import pandas as pd
import numpy as np
import csv
sys.path.append("./common")
model_save_dir = "/home/kesci/work/common/util_models/"
from utils import ReadCSV, Timer, ExtractFeature, ORI_TRAIN_NAMES, ORI_TEST_NAMES, \
ProcessChunk, AnalysisCSV, ReadCSV1
OFFLINE = False
train_name = ORI_TRAIN_NAMES
if OFFLINE:
    train_file = "/home/kesci/work/inputs/train_flast_100w.csv"
    test_file = "/home/kesci/work/inputs/train_flast_40w.csv"
    test_name = ORI_TRAIN_NAMES
else:
    train_file = "/home/kesci/work/inputs/train_first_5000w.csv"
    test_file = "/home/kesci/work/inputs/test_final_part1.csv"
    test_name = ORI_TEST_NAMES
! ls -alh /home/kesci/work/common/util_models/

total 21G
drwxr-xr-x 2 kesci root  4.0K Jul 31 14:33 .
drwxr-xr-x 6 kesci root  4.0K Jul 28 14:30 ..
-rw-r--r-- 1 kesci root   19K Jul 29 13:56 DeepFM.py
-rw-r--r-- 1 kesci users 249M Jul 22 03:01 tfidf_word_ngram-13-100w.bin
-rw-r--r-- 1 kesci users 6.2G Jul 23 17:48 tfidf_word_ngram-13.bin
-rw-r--r-- 1 kesci users 929M Jul 31 12:26 tfidf_word_ngram-13_debug_allUnique.bin
-rw-r--r-- 1 kesci users 702M Jul 31 12:36 tfidf_word_ngram-13_debug_justTrain.bin
-rw-r--r-- 1 kesci users 929M Jul 31 09:26 tfidf_word_ngram-13_debug_noUnique.bin
-rw-r--r-- 1 kesci users 1.1G Jul 31 12:53 tfidf_word_ngram-13_debug_queryConcatTitle.bin
-rw-r--r-- 1 kesci users 929M Jul 31 09:30 tfidf_word_ngram-13_debug_queryUnique.bin
-rw-r--r-- 1 kesci users 7.6G Jul 31 16:11 tfidf_word_ngram-13_online_allUnique.bin
-rw-r--r-- 1 kesci users  11M Jul 31 13:29 word2vec_debug_allUnique.kv
-rw-r--r-- 1 kesci users  70M Jul 31 13:29 word2vec_debug_allUnique.kv.vectors.npy
-rw-r--r-- 1 kesci users  11M Jul 31 13:26 wor

## 训练TFIDF

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
ngram_range, ngram_level = (1, 3), "word"
suffix = "allUnique"
if OFFLINE:
    mode = "debug"
else:
    mode = "online"
tfidf_save_file = "{}tfidf_{}_ngram-{}{}_{}_{}.bin".format(model_save_dir, ngram_level, 
    ngram_range[0], ngram_range[1], mode, suffix)
regen_tfidf_model = True
print(tfidf_save_file)

/home/kesci/work/common/util_models/tfidf_word_ngram-13_debug_queryConcatTitle.bin


In [29]:
def train_tfidf(save_file):
    word_tfidf = TfidfVectorizer(norm="l2",  # 'l2 norm' can cal similarty
                            strip_accents="unicode",
                            analyzer=ngram_level,
                            ngram_range=ngram_range,
                            use_idf=True,
                            smooth_idf=True,
                            sublinear_tf=True, min_df=2, max_df=0.9)
    with Timer("prepare corpus"):
        df_train = ReadCSV(train_file, names=train_name, iterator=False)
        df_test = ReadCSV(test_file, names=test_name,  iterator=False)
        df_data = pd.concat([df_train[['query', 'title']], df_test[['query', 'title']]], axis=0)
        del df_train, df_test; gc.collect()
        corpus =  pd.Series(
            df_data['query'].unique().tolist() +
            df_data['title'].unique().tolist()
        ).astype(str)
        print(corpus.shape)
    with Timer("train tfidf"):
        word_tfidf.fit(corpus)
    with Timer("save tfidf"):
        with open(save_file, "wb") as f:
            pickle.dump(word_tfidf, f)
            print("Model dumped to {}".format(save_file))
        del corpus
        gc.collect()
    return None

In [30]:
# 提取训练数据特征
if regen_tfidf_model:
    train_tfidf(tfidf_save_file)

----->started 'prepare corpus' block...
----->finished 'prepare corpus' block, time used:126.31s.
----->started 'train tfidf' block...
----->finished 'train tfidf' block, time used:175.6s.
----->started 'save tfidf' block...
Model dumped to /home/kesci/work/common/util_models/tfidf_word_ngram-13_debug_queryConcatTitle.bin
----->finished 'save tfidf' block, time used:66.29s.


In [9]:
# AnalysisCSV(train_file, print_rows=10, names=ORI_TRAIN_NAMES)

## 训练word2vec

In [3]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models.callbacks import CallbackAny2Vec

In [4]:
regen_word2vec = True
suffix = "concatHorizontal"
if OFFLINE:
    mode = "debug"
    word2vec_train_sentences = "/home/kesci/work/inputs/word2vec_sentences_flast_100w.txt"
else:
    mode = "online"
    word2vec_train_sentences = "/home/kesci/work/inputs/word2vec_sentences_flast_5000w.txt"
word2vec_save_name = "word2vec_{}_{}.model".format(mode, suffix)
word2vec_save_file = os.path.join(model_save_dir, word2vec_save_name)
word2vec_kv_save_name = "word2vec_{}_{}.kv".format(mode, suffix)
word2vec_kv_save_file = os.path.join(model_save_dir, word2vec_kv_save_name)

In [5]:
def PrepareWord2vecSamples(save_file):
    with Timer("prepare corpus"):
        df_train = pd.read_csv(train_file, names=train_name)
        df_test = pd.read_csv(test_file, names=test_name)
        corpus = pd.concat([df_train[['query', 'title']], df_test[['query', 'title']]], axis=0)
        del df_train, df_test
        gc.collect()
    with Timer("save sentences"):
        with open(save_file, 'w') as f:
            line_count = 0
            for i, line in corpus.iterrows():
                f.write("{} {}\n".format(line["query"], line["title"]))
                line_count += 1
                if line_count % 10000000 == 0: 
                    print(f'Processed {line_count} lines.')
            del corpus
            gc.collect()
# PrepareWord2vecSamples(word2vec_train_sentences)

----->started 'prepare corpus' block...


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


----->finished 'prepare corpus' block, time used:113.82s.
----->started 'save sentences' block...
Processed 10000000 lines.
Processed 20000000 lines.
Processed 30000000 lines.
Processed 40000000 lines.
Processed 50000000 lines.
Processed 60000000 lines.
Processed 70000000 lines.
----->finished 'save sentences' block, time used:6054.99s.


In [7]:
!head /home/kesci/work/inputs/word2vec_sentences_flast_100w.txt

844 10482 6804 8411
844 6026 15 53 100
51685 402
1913 1278 15 2215 1039
4311 456 15 321608 95
53058 31 50989 3028
236 127 1364 55882 2824
236 277 3705 2848 10035
236 160931 10592
236 68629 5975 100 29816 87051


In [8]:
class EpochSaver(CallbackAny2Vec):
    '''用于保存模型, 打印损失函数等等'''
    def __init__(self, savedir, save_name="word2vec.model"):
        os.makedirs(savedir, exist_ok=True)
        self.save_path = os.path.join(savedir, save_name)
        self.epoch = 0
        self.pre_loss = 0
        self.best_loss = 999999999.9
        self.since = time.time()

    def on_epoch_end(self, model):
        self.epoch += 1
        cum_loss = model.get_latest_training_loss() # 返回的是从第一个epoch累计的
        epoch_loss = cum_loss - self.pre_loss
        time_taken = time.time() - self.since
        print("Epoch %d, loss: %.2f, time: %dmin %ds" % 
                    (self.epoch, epoch_loss, time_taken//60, time_taken%60))
        if self.best_loss > epoch_loss and epoch_loss > 0:
            self.best_loss = epoch_loss
            print("Better model. Best loss: %.2f" % self.best_loss)
            model.save(self.save_path)
            print("Model %s save done!" % self.save_path)
        self.pre_loss = cum_loss
        self.since = time.time()

In [9]:
if regen_word2vec:
    # 1, 构建模型(不训练)
    model_word2vec = Word2Vec(min_count=2, 
                              window=10, 
                              size=100,
                              workers=10)

In [10]:
if regen_word2vec:
    # 2, 遍历一遍语料库
    since = time.time()
    model_word2vec.build_vocab(
        LineSentence(word2vec_train_sentences)
    )
    time_elapsed = time.time() - since
    print('Time to build vocab: {:.0f}min {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Time to build vocab: 7min 58s


In [7]:
if regen_word2vec:
    # 3, 训练
    since = time.time()
    model_word2vec.train(
        LineSentence(word2vec_train_sentences), 
        total_examples=model_word2vec.corpus_count, 
        epochs=30, compute_loss=True, report_delay=60*10,
        callbacks=[EpochSaver(model_save_dir, word2vec_save_name)])
    time_elapsed = time.time() - since
    print('Time to train: {:.0f}min {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

Epoch 1, loss: 69566760.00, time: 18min 33s
Better model. Best loss: 69566760.00
Model /home/kesci/work/common/util_models/word2vec_online_concatHorizontal.model save done!
Epoch 2, loss: 23542272.00, time: 18min 24s
Better model. Best loss: 23542272.00
Model /home/kesci/work/common/util_models/word2vec_online_concatHorizontal.model save done!
Epoch 3, loss: 24322496.00, time: 18min 20s
Epoch 4, loss: 16786200.00, time: 18min 19s
Better model. Best loss: 16786200.00
Model /home/kesci/work/common/util_models/word2vec_online_concatHorizontal.model save done!
Epoch 5, loss: 0.00, time: 18min 17s
Epoch 6, loss: 0.00, time: 18min 18s
Epoch 7, loss: 0.00, time: 18min 20s
Epoch 8, loss: 0.00, time: 18min 20s
Epoch 9, loss: 0.00, time: 18min 19s
Epoch 10, loss: 0.00, time: 18min 17s
Epoch 11, loss: 0.00, time: 18min 19s
Epoch 12, loss: 0.00, time: 18min 20s
Epoch 13, loss: 0.00, time: 18min 20s
Epoch 14, loss: 0.00, time: 18min 20s
Epoch 15, loss: 0.00, time: 18min 20s
Epoch 16, loss: 0.00, ti

KeyboardInterrupt: 

In [12]:
if regen_word2vec:
    tmp_model = Word2Vec.load(word2vec_save_file)
    # 这个Loss是累积的
    print("Loss:", tmp_model.get_latest_training_loss())
    tmp_model.wv.save(word2vec_kv_save_file)
    print("word2vec kv saved to {}".format(word2vec_kv_save_file))
    del tmp_model
    gc.collect()

Loss: 134217728.0
word2vec kv saved to /home/kesci/work/common/util_models/word2vec_online_concatHorizontal.kv


In [13]:
# tmp_model = Word2Vec.load(word2vec_save_file)
# print(tmp_model["10"])

## 杂项

In [1]:
import os
print(os.getpid())

637


In [ ]:
! kill -9 637